In [125]:
import pandas as pd
import json
import re
from constants import congress_map, funds_map
from dataclasses import dataclass
from dotenv import load_dotenv
import os
import requests

In [126]:
df = pd.read_pickle("filings.pkl")
df.drop(0, axis=0, inplace=True)

In [127]:
load_dotenv()  # Load environment variables from .env file
API_TOKEN = os.getenv("CONGRESS_API_TOKEN")

In [128]:
df.drop(["url", "filing_document_url", "termination_date",  "registrant_address_1", "registrant_address_2", "registrant_different_address", "registrant_zip", "conviction_disclosures", "affiliated_organizations", "foreign_entities","registrant_state", "posted_by_name", "registrant_country", "registrant_ppb_country", "registrant_city", "filing_period_display", "filing_type", "filing_type_display", "filing_document_content_type", "expenses_method", "expenses_method_display", "registrant"], axis=1, inplace=True, errors='ignore')
df["congress"] = df.filing_year.map(congress_map)
df

,index,filing_uuid,filing_year,filing_period,income,expenses,dt_posted,client,lobbying_activities,page,congress
1,2,2aeb2b4f-41a4-4e76-9a80-3cd916e388f2,2015,first_quarter,None,900000.00,2015-04-20T17:23:29.953000-04:00,"{'id': 133239, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'FOR', 'general_issue_...",1,114
2,3,13252aa4-85a9-4557-b77d-a7de6c11f2c4,2015,first_quarter,None,1029000.00,2015-07-14T14:12:13.600000-04:00,"{'id': 133239, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'TAX', 'general_issue_...",1,114
3,4,7c555d5e-d44d-4253-820b-1b4838f22f51,2015,second_quarter,30000.00,None,2015-07-17T12:01:49.617000-04:00,"{'id': 123378, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'DEF', 'general_issue_...",1,114
4,5,6d0f0a73-42fc-4a2b-9e0f-d24a286bde4f,2015,second_quarter,None,550000.00,2015-07-20T15:02:39.627000-04:00,"{'id': 133239, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'CHM', 'general_issue_...",1,114
5,6,babd0ecd-9b3c-4b35-93ec-80fdb1694989,2015,third_quarter,30000.00,None,2015-10-20T11:49:32.800000-04:00,"{'id': 123378, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'DEF', 'general_issue_...",1,114
...,...,...,...,...,...,...,...,...,...,...,...
6038,6039,5f45a04d-c3bd-445c-9911-6cd88082e740,2025,second_quarter,60000.00,None,2025-07-21T11:59:30-04:00,"{'id': 205812, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'ADV', 'general_issue_...",1,119
6039,6040,2b2932a5-dfa7-40d1-aa0a-9fbef7747040,2025,second_quarter,None,180000.00,2025-07-21T18:21:27-04:00,"{'id': 213197, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'MIA', 'general_issue_...",1,119
6040,6041,04b0c3fe-213b-4662-ad15-f855f9cca7e8,2025,third_quarter,60000.00,None,2025-10-20T13:31:03-04:00,"{'id': 205812, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'ADV', 'general_issue_...",1,119
6041,6042,5ac1d56b-9c1f-42eb-bce4-c2152e5ec107,2025,third_quarter,60000.00,None,2025-10-20T14:47:54-04:00,"{'id': 205812, 'url': 'https://lda.senate.gov/...","[{'general_issue_code': 'ADV', 'general_issue_...",1,119


In [129]:
for key in df.client[1].keys():
    if key in ("id", "name", "general_description", "country"):
        df[f"client_{key}"] = df.client.apply(lambda x: x.get(key, None))
df.drop("client", axis=1, inplace=True)
df

,index,filing_uuid,filing_year,filing_period,income,expenses,dt_posted,lobbying_activities,page,congress,client_id,client_name,client_general_description,client_country
1,2,2aeb2b4f-41a4-4e76-9a80-3cd916e388f2,2015,first_quarter,None,900000.00,2015-04-20T17:23:29.953000-04:00,"[{'general_issue_code': 'FOR', 'general_issue_...",1,114,133239,3M COMPANY,Federal Lobbying Activity,US
2,3,13252aa4-85a9-4557-b77d-a7de6c11f2c4,2015,first_quarter,None,1029000.00,2015-07-14T14:12:13.600000-04:00,"[{'general_issue_code': 'TAX', 'general_issue_...",1,114,133239,3M COMPANY,Federal Lobbying Activity,US
3,4,7c555d5e-d44d-4253-820b-1b4838f22f51,2015,second_quarter,30000.00,None,2015-07-17T12:01:49.617000-04:00,"[{'general_issue_code': 'DEF', 'general_issue_...",1,114,123378,3M COMPANY,None,US
4,5,6d0f0a73-42fc-4a2b-9e0f-d24a286bde4f,2015,second_quarter,None,550000.00,2015-07-20T15:02:39.627000-04:00,"[{'general_issue_code': 'CHM', 'general_issue_...",1,114,133239,3M COMPANY,Federal Lobbying Activity,US
5,6,babd0ecd-9b3c-4b35-93ec-80fdb1694989,2015,third_quarter,30000.00,None,2015-10-20T11:49:32.800000-04:00,"[{'general_issue_code': 'DEF', 'general_issue_...",1,114,123378,3M COMPANY,None,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6038,6039,5f45a04d-c3bd-445c-9911-6cd88082e740,2025,second_quarter,60000.00,None,2025-07-21T11:59:30-04:00,"[{'general_issue_code': 'ADV', 'general_issue_...",1,119,205812,"YAHOO INC, AND VAR. SUBS/AFFILIATES (FKA COLLE...",Internet/search/online ad technology company,US
6039,6040,2b2932a5-dfa7-40d1-aa0a-9fbef7747040,2025,second_quarter,None,180000.00,2025-07-21T18:21:27-04:00,"[{'general_issue_code': 'MIA', 'general_issue_...",1,119,213197,YAHOO INC. AND VAR. SUBS/AFFILIATES,None,US
6040,6041,04b0c3fe-213b-4662-ad15-f855f9cca7e8,2025,third_quarter,60000.00,None,2025-10-20T13:31:03-04:00,"[{'general_issue_code': 'ADV', 'general_issue_...",1,119,205812,"YAHOO INC, AND VAR. SUBS/AFFILIATES (FKA COLLE...",Internet/search/online ad technology company,US
6041,6042,5ac1d56b-9c1f-42eb-bce4-c2152e5ec107,2025,third_quarter,60000.00,None,2025-10-20T14:47:54-04:00,"[{'general_issue_code': 'ADV', 'general_issue_...",1,119,205812,"YAHOO INC, AND VAR. SUBS/AFFILIATES (FKA COLLE...",Internet/search/online ad technology company,US


In [130]:
def parse_lobbyist_activities(activity_list):
    for activity in activity_list:
        activity.pop("lobbyists", None)
        activity.pop("government_entities", None)
        activity.pop("foreign_entity_issues", None)
        description = activity.get("description", "")
        matches = set()
        for mather in [r"(H\. *R\. *\d+)", r"(S\. *\d+)", r"(S\. *J\. *Res\. *\d+)", r"(H\. *J\. *Res\. *\d+)", r"(H\. *Con\. *Res\. *\d+)", r"(S\. *Con\. *Res\. *\d+)", r"(H\. *Res\. *\d+)", r"(S\. *Res\. *\d+)"]:
            m = re.findall(mather, description)
            for match in m:
                matches.add(match)
        activity["bills"] = [match.replace(" ", "") for match in matches]
    return activity_list

In [131]:
df.lobbying_activities = df.lobbying_activities.apply(parse_lobbyist_activities)
df

,index,filing_uuid,filing_year,filing_period,income,expenses,dt_posted,lobbying_activities,page,congress,client_id,client_name,client_general_description,client_country
1,2,2aeb2b4f-41a4-4e76-9a80-3cd916e388f2,2015,first_quarter,None,900000.00,2015-04-20T17:23:29.953000-04:00,"[{'general_issue_code': 'FOR', 'general_issue_...",1,114,133239,3M COMPANY,Federal Lobbying Activity,US
2,3,13252aa4-85a9-4557-b77d-a7de6c11f2c4,2015,first_quarter,None,1029000.00,2015-07-14T14:12:13.600000-04:00,"[{'general_issue_code': 'TAX', 'general_issue_...",1,114,133239,3M COMPANY,Federal Lobbying Activity,US
3,4,7c555d5e-d44d-4253-820b-1b4838f22f51,2015,second_quarter,30000.00,None,2015-07-17T12:01:49.617000-04:00,"[{'general_issue_code': 'DEF', 'general_issue_...",1,114,123378,3M COMPANY,None,US
4,5,6d0f0a73-42fc-4a2b-9e0f-d24a286bde4f,2015,second_quarter,None,550000.00,2015-07-20T15:02:39.627000-04:00,"[{'general_issue_code': 'CHM', 'general_issue_...",1,114,133239,3M COMPANY,Federal Lobbying Activity,US
5,6,babd0ecd-9b3c-4b35-93ec-80fdb1694989,2015,third_quarter,30000.00,None,2015-10-20T11:49:32.800000-04:00,"[{'general_issue_code': 'DEF', 'general_issue_...",1,114,123378,3M COMPANY,None,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6038,6039,5f45a04d-c3bd-445c-9911-6cd88082e740,2025,second_quarter,60000.00,None,2025-07-21T11:59:30-04:00,"[{'general_issue_code': 'ADV', 'general_issue_...",1,119,205812,"YAHOO INC, AND VAR. SUBS/AFFILIATES (FKA COLLE...",Internet/search/online ad technology company,US
6039,6040,2b2932a5-dfa7-40d1-aa0a-9fbef7747040,2025,second_quarter,None,180000.00,2025-07-21T18:21:27-04:00,"[{'general_issue_code': 'MIA', 'general_issue_...",1,119,213197,YAHOO INC. AND VAR. SUBS/AFFILIATES,None,US
6040,6041,04b0c3fe-213b-4662-ad15-f855f9cca7e8,2025,third_quarter,60000.00,None,2025-10-20T13:31:03-04:00,"[{'general_issue_code': 'ADV', 'general_issue_...",1,119,205812,"YAHOO INC, AND VAR. SUBS/AFFILIATES (FKA COLLE...",Internet/search/online ad technology company,US
6041,6042,5ac1d56b-9c1f-42eb-bce4-c2152e5ec107,2025,third_quarter,60000.00,None,2025-10-20T14:47:54-04:00,"[{'general_issue_code': 'ADV', 'general_issue_...",1,119,205812,"YAHOO INC, AND VAR. SUBS/AFFILIATES (FKA COLLE...",Internet/search/online ad technology company,US


In [132]:
@dataclass(frozen=True)
class Bill:
    bill_type: str
    number: int
    congress: int

In [133]:
bills: set[Bill] = set()
for congress, activities in zip(df.congress, df.lobbying_activities, strict=True):
    for activity in activities:
        for bill in activity.get("bills", []):
            components = bill.split(".")
            t = "".join(components[:-1]).lower()
            n = int(components[-1])
            bills.add(Bill(bill_type=t, number=n, congress=congress))
bills = list(bills)
bills

[Bill(bill_type='s', number=934, congress=115),
 Bill(bill_type='s', number=1068, congress=115),
 Bill(bill_type='s', number=3449, congress=116),
 Bill(bill_type='hr', number=1074, congress=117),
 Bill(bill_type='s', number=2708, congress=118),
 Bill(bill_type='hr', number=5022, congress=114),
 Bill(bill_type='hr', number=2028, congress=115),
 Bill(bill_type='s', number=926, congress=116),
 Bill(bill_type='hr', number=7891, congress=118),
 Bill(bill_type='hr', number=4198, congress=118),
 Bill(bill_type='s', number=702, congress=115),
 Bill(bill_type='s', number=1318, congress=118),
 Bill(bill_type='hr', number=4528, congress=114),
 Bill(bill_type='hr', number=1534, congress=115),
 Bill(bill_type='hr', number=7222, congress=117),
 Bill(bill_type='hr', number=7659, congress=118),
 Bill(bill_type='hr', number=5003, congress=114),
 Bill(bill_type='s', number=1431, congress=116),
 Bill(bill_type='s', number=1169, congress=116),
 Bill(bill_type='hr', number=7435, congress=117),
 Bill(bill_t

In [134]:
len(bills)

6902

In [135]:
def fetch_cosponsors(bill: Bill | str) -> list[dict]:
    if bill == "":
        return []
    if isinstance(bill, Bill):
        url = f"https://api.congress.gov/v3/bill/{bill.congress}/{bill.bill_type}/{bill.number}/cosponsors?api_key={API_TOKEN}&limit=250"
    else:
        url = f"{bill}&api_key={API_TOKEN}&limit=250"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return data.get("cosponsors", [])

In [136]:
def fetch_summaries(bill: Bill | str) -> list[dict]:
    if bill == "":
        return []
    if isinstance(bill, Bill):
        url = f"https://api.congress.gov/v3/bill/{bill.congress}/{bill.bill_type}/{bill.number}/cosponsors?api_key={API_TOKEN}&limit=250"
    else:
        url = f"{bill}&api_key={API_TOKEN}&limit=250"
    try:
        response = requests.get(url)
        data = response.json()
        summaries = data.get("summaries", [])
        if len(summaries) > 0:
            return summaries[0].get("text", "")
        else:
            return ""
    except requests.RequestException as e:
        print(f"Error fetching summaries: {e}")
        return ""

In [137]:
def fetch_bills(bills: list[Bill], already_fetched: set[Bill] = set()) -> list[dict]:
    for bill in bills:
        if bill in already_fetched:
            print(f"Skipping already fetched bill {bill.bill_type} {bill.number} from congress {bill.congress}")
            continue
        else:
            print(f"Fetching bill {bill.bill_type} {bill.number} from congress {bill.congress}")
        response = requests.get(f"https://api.congress.gov/v3/bill/{bill.congress}/{bill.bill_type}/{bill.number}?api_key={API_TOKEN}")
        if response.status_code == 200:
            data = response.json()
            data = data["bill"]
            data["cosponsors_d"] = 0
            data["cosponsors_r"] = 0
            for cosponsor in fetch_cosponsors(data.get("cosponsors", {}).get("url", "")):
                if cosponsor.get("party", "") == "D":
                    data["cosponsors_d"] += 1
                elif cosponsor.get("party", "") == "R":
                    data["cosponsors_r"] += 1
            yield data
        else:
            print(f"Failed to fetch bill {bill.bill_type} {bill.number} from congress {bill.congress}: {response.status_code}")

In [138]:
bills_df = pd.read_pickle("bills.pkl")
already_fetched_bills = set()
for _, row in bills_df.iterrows():
    already_fetched_bills.add(Bill(bill_type=row.type.lower(), number=row.number, congress=row.congress))
for fetched_bills in fetch_bills(bills, already_fetched=already_fetched_bills):
    bills_df = pd.concat([bills_df, pd.json_normalize(fetched_bills)], ignore_index=True)
    bills_df.type = bills_df.type.astype("category")
    bills_df.congress = bills_df.congress.astype(int)
    bills_df.number = bills_df.number.astype(int)
    bills_df.drop_duplicates(subset=["type", "number", "congress"], inplace=True)
    bills_df.to_pickle("bills.pkl")

Skipping already fetched bill s 934 from congress 115
Skipping already fetched bill s 1068 from congress 115
Skipping already fetched bill s 3449 from congress 116
Skipping already fetched bill hr 1074 from congress 117
Skipping already fetched bill s 2708 from congress 118
Skipping already fetched bill hr 5022 from congress 114
Skipping already fetched bill hr 2028 from congress 115
Skipping already fetched bill s 926 from congress 116
Skipping already fetched bill hr 7891 from congress 118
Skipping already fetched bill hr 4198 from congress 118
Skipping already fetched bill s 702 from congress 115
Skipping already fetched bill s 1318 from congress 118
Skipping already fetched bill hr 4528 from congress 114
Skipping already fetched bill hr 1534 from congress 115
Skipping already fetched bill hr 7222 from congress 117
Skipping already fetched bill hr 7659 from congress 118
Skipping already fetched bill hr 5003 from congress 114
Skipping already fetched bill s 1431 from congress 116
Fet

KeyboardInterrupt: 

In [ ]:
len(bills_df)

2472

## Process

In [ ]:
bills_df: pd.DataFrame = pd.read_pickle("bills_summary.pkl")
for idx, row in bills_df.iterrows():
    if row["summaries.url"] and row["summary"] == "":
        bills_df.loc[idx, 'summary'] = fetch_summaries(row["summaries.url"])
        bills_df.drop_duplicates(subset=["type", "number", "congress"], inplace=True)
        bills_df.to_pickle("bills_summary.pkl")

Error fetching summaries: Invalid URL 'nan&api_key=mi2jdMdZNMmUPpsk8Oqo2nrRaZL8uxRfNX2Pslgs&limit=250': No scheme supplied. Perhaps you meant https://nan&api_key=mi2jdMdZNMmUPpsk8Oqo2nrRaZL8uxRfNX2Pslgs&limit=250?
Error fetching summaries: Invalid URL 'nan&api_key=mi2jdMdZNMmUPpsk8Oqo2nrRaZL8uxRfNX2Pslgs&limit=250': No scheme supplied. Perhaps you meant https://nan&api_key=mi2jdMdZNMmUPpsk8Oqo2nrRaZL8uxRfNX2Pslgs&limit=250?
Error fetching summaries: Invalid URL 'nan&api_key=mi2jdMdZNMmUPpsk8Oqo2nrRaZL8uxRfNX2Pslgs&limit=250': No scheme supplied. Perhaps you meant https://nan&api_key=mi2jdMdZNMmUPpsk8Oqo2nrRaZL8uxRfNX2Pslgs&limit=250?
Error fetching summaries: Invalid URL 'nan&api_key=mi2jdMdZNMmUPpsk8Oqo2nrRaZL8uxRfNX2Pslgs&limit=250': No scheme supplied. Perhaps you meant https://nan&api_key=mi2jdMdZNMmUPpsk8Oqo2nrRaZL8uxRfNX2Pslgs&limit=250?
Error fetching summaries: Invalid URL 'nan&api_key=mi2jdMdZNMmUPpsk8Oqo2nrRaZL8uxRfNX2Pslgs&limit=250': No scheme supplied. Perhaps you meant ht

KeyboardInterrupt: 

In [ ]:
bills_df[bills_df["summary"] == ""]

,congress,constitutionalAuthorityStatementText,introducedDate,legislationUrl,number,originChamber,originChamberCode,sponsors,title,type,...,titles.url,cboCostEstimates,laws,committeeReports,amendments.count,amendments.url,latestAction.actionTime,notes,onBehalfOfSponsor,summary
22,118,"<pre>\n[Congressional Record Volume 170, Numbe...",2024-03-12,https://www.congress.gov/bill/118th-congress/h...,7624,House,H,"[{'bioguideId': 'C001066', 'district': 14, 'fi...",Advancing GETs Act of 2024,HR,...,https://api.congress.gov/v3/bill/118/hr/7624/t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
44,118,NaN,2023-09-13,https://www.congress.gov/bill/118th-congress/s...,2780,Senate,S,"[{'bioguideId': 'H001076', 'firstName': 'Maggi...",Medication Affordability and Patent Integrity Act,S,...,https://api.congress.gov/v3/bill/118/s/2780/ti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
67,118,NaN,2023-11-07,https://www.congress.gov/bill/118th-congress/s...,3236,Senate,S,"[{'bioguideId': 'W000800', 'firstName': 'Peter...",Emergency Medical Services Reimbursement for O...,S,...,https://api.congress.gov/v3/bill/118/s/3236/ti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
83,118,NaN,2024-07-09,https://www.congress.gov/bill/118th-congress/s...,4649,Senate,S,"[{'bioguideId': 'L000577', 'firstName': 'Mike'...",A bill to amend the Mutual Security Act of 195...,S,...,https://api.congress.gov/v3/bill/118/s/4649/ti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
120,118,NaN,2023-06-15,https://www.congress.gov/bill/118th-congress/s...,2033,Senate,S,"[{'bioguideId': 'K000367', 'firstName': 'Amy',...",American Innovation and Choice Online Act,S,...,https://api.congress.gov/v3/bill/118/s/2033/ti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,118,NaN,2023-01-30,https://www.congress.gov/bill/118th-congress/s...,142,Senate,S,"[{'bioguideId': 'K000367', 'firstName': 'Amy',...",Preserve Access to Affordable Generics and Bio...,S,...,https://api.congress.gov/v3/bill/118/s/142/tit...,[{'description': 'As reported by the Senate Co...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2468,118,NaN,2023-07-10,https://www.congress.gov/bill/118th-congress/s...,2220,Senate,S,"[{'bioguideId': 'C001088', 'firstName': 'Chris...",PREVAIL Act,S,...,https://api.congress.gov/v3/bill/118/s/2220/ti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2469,116,"<pre>\n[Congressional Record Volume 166, Numbe...",2020-06-01,https://www.congress.gov/bill/116th-congress/h...,7079,House,H,"[{'bioguideId': 'L000585', 'district': 18, 'fi...",Clean Start: Back to Work Tax Credit Act,HR,...,https://api.congress.gov/v3/bill/116/hr/7079/t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2470,115,NaN,2017-11-28,https://www.congress.gov/bill/115th-congress/s...,1,Senate,S,"[{'bioguideId': 'E000285', 'firstName': 'MICHA...",An original bill to provide for reconciliation...,S,...,https://api.congress.gov/v3/bill/115/s/1/title...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [ ]:
bills_df: pd.DataFrame = pd.read_pickle("bills.pkl")
# bills_df.drop(columns=["constitutionalAuthorityStatementText", "textVersions.url", "textVersions.count", "cosponsors.url", "relatedBills.url", "titles.count", "titles.url", "cosponsors.count", "cosponsors.countIncludingWithdrawnCosponsors", "summaries.count", "summaries.url", "subjects.count"], inplace=True, errors='ignore')
bills_df

,congress,constitutionalAuthorityStatementText,introducedDate,legislationUrl,number,originChamber,originChamberCode,sponsors,title,type,...,titles.url,cboCostEstimates,laws,committeeReports,amendments.count,amendments.url,latestAction.actionTime,notes,onBehalfOfSponsor,summary
0,114,"<pre>[Congressional Record Volume 161, Number ...",2015-06-11,https://www.congress.gov/bill/114th-congress/h...,2734,House,H,"[{'bioguideId': 'B000213', 'district': 6, 'fir...",Do Not Track Kids Act of 2015,HR,...,https://api.congress.gov/v3/bill/114/hr/2734/t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,115,NaN,2017-03-01,https://www.congress.gov/bill/115th-congress/s...,479,Senate,S,"[{'bioguideId': 'B000944', 'firstName': 'Sherr...",Removing Barriers to Colorectal Cancer Screeni...,S,...,https://api.congress.gov/v3/bill/115/s/479/tit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,118,"<pre>\n[Congressional Record Volume 169, Numbe...",2023-05-23,https://www.congress.gov/bill/118th-congress/h...,3603,House,H,"[{'bioguideId': 'C001121', 'district': 6, 'fir...",Small Business Innovation Voucher Act of 2023,HR,...,https://api.congress.gov/v3/bill/118/hr/3603/t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,115,<pre>\nFrom the Congressional Record Online th...,2018-12-12,https://www.congress.gov/bill/115th-congress/h...,7289,House,H,"[{'bioguideId': 'S001195', 'district': 8, 'fir...",Fairness for Every Driver Act,HR,...,https://api.congress.gov/v3/bill/115/hr/7289/t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,117,NaN,2022-06-13,https://www.congress.gov/bill/117th-congress/s...,4380,Senate,S,"[{'bioguideId': 'W000817', 'firstName': 'Eliza...",Military Vehicle Fleet Electrification Act,S,...,https://api.congress.gov/v3/bill/117/s/4380/ti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,118,NaN,2023-04-26,https://www.congress.gov/bill/118th-congress/s...,1318,Senate,S,"[{'bioguideId': 'K000367', 'firstName': 'Amy',...",Election Worker Protection Act of 2023,S,...,https://api.congress.gov/v3/bill/118/s/1318/ti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2483,114,"<pre>[Congressional Record Volume 162, Number ...",2016-02-10,https://www.congress.gov/bill/114th-congress/h...,4528,House,H,"[{'bioguideId': 'L000582', 'district': 33, 'fi...",ENCRYPT Act of 2016,HR,...,https://api.congress.gov/v3/bill/114/hr/4528/t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2484,115,"<pre>[Congressional Record Volume 163, Number ...",2017-03-15,https://www.congress.gov/bill/115th-congress/h...,1534,House,H,"[{'bioguideId': 'D000624', 'district': 12, 'fi...",21st Century Worker Opportunity Act,HR,...,https://api.congress.gov/v3/bill/115/hr/1534/t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2485,118,"<pre>\n[Congressional Record Volume 170, Numbe...",2024-03-13,https://www.congress.gov/bill/118th-congress/h...,7659,House,H,"[{'bioguideId': 'G000546', 'district': 6, 'fir...",Coast Guard Authorization Act of 2024,HR,...,https://api.congress.gov/v3/bill/118/hr/7659/t...,[{'description': 'As reported by the House Com...,NaN,"[{'citation': 'H. Rept. 118-495', 'url': 'http...",NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
bills_df.columns

Index(['congress', 'constitutionalAuthorityStatementText', 'introducedDate',
       'legislationUrl', 'number', 'originChamber', 'originChamberCode',
       'sponsors', 'title', 'type', 'updateDate', 'updateDateIncludingText',
       'cosponsors_d', 'cosponsors_r', 'actions.count', 'actions.url',
       'committees.count', 'committees.url', 'cosponsors.count',
       'cosponsors.countIncludingWithdrawnCosponsors', 'cosponsors.url',
       'latestAction.actionDate', 'latestAction.text', 'policyArea.name',
       'relatedBills.count', 'relatedBills.url', 'subjects.count',
       'subjects.url', 'summaries.count', 'summaries.url',
       'textVersions.count', 'textVersions.url', 'titles.count', 'titles.url',
       'cboCostEstimates', 'laws', 'committeeReports', 'amendments.count',
       'amendments.url', 'latestAction.actionTime', 'notes',
       'onBehalfOfSponsor', 'summary'],
      dtype='object')

In [ ]:
bills_df["passed"] = (bills_df["latestAction.text"].str.lower().str.contains("public law")) | (bills_df["latestAction.text"].str.lower().str.contains("enacted") | (bills_df["latestAction.text"].str.lower().str.contains("by president")))
bills_df["passed"].value_counts()

passed
False    2394
True       93
Name: count, dtype: int64

In [ ]:
bills_df.to_pickle("bills_processed.pkl")

In [ ]:
funds_map = {
    # 🔬 HEALTH & LIFE SCIENCES
    "Health": "XLV",  # Health Care Select Sector SPDR
    "Social Welfare": "XLV",  # includes Medicaid / social programs
    "Education": "XLV",  # overlaps with healthcare/social spending sometimes

    # 💰 ECONOMY, TAX, FINANCE
    "Economics and Public Finance": "XLF",  # Finance sector
    "Finance and Financial Sector": "XLF",  # Banking, insurance, lending
    "Taxation": "XLF",  # Tax policies influence financial markets
    "Foreign Trade and International Finance": "XLF",
    "Commerce": "XLY",  # Consumer discretionary / commerce
    "Labor and Employment": "XLY",  # Consumer spending tied to jobs

    # ⚙️ INDUSTRIALS / INFRASTRUCTURE
    "Transportation and Public Works": "XLI",  # Industrials sector
    "Energy": "XLE",  # Energy Select Sector SPDR
    "Water Resources Development": "XLU",  # Utilities sector (public water/power)
    "Environmental Protection": "ICLN",  # Clean energy / environment ETF
    "Public Lands and Natural Resources": "XLI",  # Construction / land management
    "Infrastructure": "XLI",

    # 💻 TECHNOLOGY & INNOVATION
    "Science, Technology, Communications": "XLK",  # Technology
    "International Affairs": "SPY",  # Broad-market (macro policy)
    "Telecommunications": "XLC",  # Communications Services ETF

    # 🛡️ DEFENSE & SECURITY
    "Armed Forces and National Security": "XAR",  # Aerospace & defense ETF
    "Crime and Law Enforcement": "XAR",  # Defense/security services
    "Emergency Management": "XAR",  # National security coordination

    # 🌾 AGRICULTURE / FOOD / CONSUMER
    "Agriculture and Food": "MOO",  # Agribusiness ETF
    "Animals": "MOO",  # Agriculture/animal welfare impacts food production
    "Families": "XLP",  # Consumer staples (household consumption)
    "Housing and Community Development": "XLRE",  # Real estate ETF

    # 🌎 ENVIRONMENT & RESOURCES
    "Environmental Protection": "ICLN",  # Clean energy
    "Public Lands and Natural Resources": "XLI",
    "Water Resources Development": "XLU",

    # ⚖️ GOVERNMENT & POLITICS
    "Government Operations and Politics": "SPY",  # Broad S&P 500
    "Congress": "SPY",  # Neutral (political operations, broad impact)
    "Law": "SPY",  # Legal / judiciary not tied to a single sector

    # 🌍 INTERNATIONAL
    "International Affairs": "SPY",  # Broad international relations
    "Foreign Trade and International Finance": "XLF",

    # 🏠 SOCIAL & DOMESTIC ISSUES
    "Civil Rights and Liberties, Minority Issues": "XLP",  # Social services impact consumer staples
    "Native Americans": "XLP",  # Community development focus
    "Arts, Culture, Religion": "XLY",  # Consumer discretionary
}

bills_df: pd.DataFrame = pd.read_pickle("bills_processed.pkl")
bills_df["funds"] = bills_df["policyArea.name"].apply(lambda x: funds_map.get(x, "SPY"))
bills_df["funds"].value_counts()

funds
XLF     529
XLV     429
XLK     308
SPY     260
XLE     206
XLY     205
XAR     197
XLI     156
ICLN    117
MOO      43
XLP      16
XLU      14
XLRE      7
Name: count, dtype: int64

In [ ]:
import yfinance as yf
import datetime
funds_map_etfs = {ticker: yf.Ticker(ticker) for ticker in funds_map.values()}

tradings = []
for index, row in bills_df.iterrows():
    ticker = funds_map_etfs.get(row.funds, None)
    if ticker is None:
        tradings.append(None)
        continue
    approval_date = row["latestAction.actionDate"]
    date_1 = datetime.datetime.strptime(approval_date, "%Y-%m-%d").date()
    before_date = date_1 - datetime.timedelta(days=30)
    tradings.append(ticker.history(start=before_date, end=approval_date, interval="1d"))
bills_df["tradings"] = tradings

$XLRE: possibly delisted; no price data found  (1d 2015-02-17 -> 2015-03-19) (Yahoo error = "Data doesn't exist for startDate = 1424149200, endDate = 1426737600")


In [ ]:
tradings_bills_df = bills_df[bills_df["tradings"] != None]
tradings_bills_df

,congress,constitutionalAuthorityStatementText,introducedDate,legislationUrl,number,originChamber,originChamberCode,sponsors,title,type,...,committeeReports,amendments.count,amendments.url,latestAction.actionTime,notes,onBehalfOfSponsor,summary,passed,funds,tradings
0,114,"<pre>[Congressional Record Volume 161, Number ...",2015-06-11,https://www.congress.gov/bill/114th-congress/h...,2734,House,H,"[{'bioguideId': 'B000213', 'district': 6, 'fir...",Do Not Track Kids Act of 2015,HR,...,NaN,NaN,NaN,NaN,NaN,NaN,,False,XLY,Open Hig...
1,115,NaN,2017-03-01,https://www.congress.gov/bill/115th-congress/s...,479,Senate,S,"[{'bioguideId': 'B000944', 'firstName': 'Sherr...",Removing Barriers to Colorectal Cancer Screeni...,S,...,NaN,NaN,NaN,NaN,NaN,NaN,,False,XLV,Open Hig...
2,118,"<pre>\n[Congressional Record Volume 169, Numbe...",2023-05-23,https://www.congress.gov/bill/118th-congress/h...,3603,House,H,"[{'bioguideId': 'C001121', 'district': 6, 'fir...",Small Business Innovation Voucher Act of 2023,HR,...,NaN,NaN,NaN,NaN,NaN,NaN,,False,XLY,Open H...
3,115,<pre>\nFrom the Congressional Record Online th...,2018-12-12,https://www.congress.gov/bill/115th-congress/h...,7289,House,H,"[{'bioguideId': 'S001195', 'district': 8, 'fir...",Fairness for Every Driver Act,HR,...,NaN,NaN,NaN,NaN,NaN,NaN,,False,XLF,Open Hig...
4,117,NaN,2022-06-13,https://www.congress.gov/bill/117th-congress/s...,4380,Senate,S,"[{'bioguideId': 'W000817', 'firstName': 'Eliza...",Military Vehicle Fleet Electrification Act,S,...,NaN,NaN,NaN,NaN,NaN,NaN,,False,XAR,Open H...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,118,NaN,2023-04-26,https://www.congress.gov/bill/118th-congress/s...,1318,Senate,S,"[{'bioguideId': 'K000367', 'firstName': 'Amy',...",Election Worker Protection Act of 2023,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,SPY,Open H...
2483,114,"<pre>[Congressional Record Volume 162, Number ...",2016-02-10,https://www.congress.gov/bill/114th-congress/h...,4528,House,H,"[{'bioguideId': 'L000582', 'district': 33, 'fi...",ENCRYPT Act of 2016,HR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,XAR,Open Hig...
2484,115,"<pre>[Congressional Record Volume 163, Number ...",2017-03-15,https://www.congress.gov/bill/115th-congress/h...,1534,House,H,"[{'bioguideId': 'D000624', 'district': 12, 'fi...",21st Century Worker Opportunity Act,HR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,XLY,Open Hig...
2485,118,"<pre>\n[Congressional Record Volume 170, Numbe...",2024-03-13,https://www.congress.gov/bill/118th-congress/h...,7659,House,H,"[{'bioguideId': 'G000546', 'district': 6, 'fir...",Coast Guard Authorization Act of 2024,HR,...,"[{'citation': 'H. Rept. 118-495', 'url': 'http...",NaN,NaN,NaN,NaN,NaN,NaN,False,XLI,Open H...


In [ ]:
tradings_bills_df.cosponsors_d

0       12
1       39
2        1
3        0
4        5
        ..
2482    27
2483     5
2484     0
2485     2
2486     1
Name: cosponsors_d, Length: 2487, dtype: int64

In [ ]:
tradings_bills_df["tradings"].iloc[0]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2015-05-13 00:00:00-04:00,67.770408,68.001762,67.254297,67.307686,7921600,0.0,0.0,0.0
2015-05-14 00:00:00-04:00,67.565727,67.699205,67.200888,67.610222,6830300,0.0,0.0,0.0
2015-05-15 00:00:00-04:00,67.690323,68.153038,67.619134,68.153038,7189900,0.0,0.0,0.0
2015-05-18 00:00:00-04:00,68.037347,68.535657,68.019553,68.411079,3961500,0.0,0.0,0.0
2015-05-19 00:00:00-04:00,68.597955,68.740333,68.322106,68.393295,4645400,0.0,0.0,0.0
2015-05-20 00:00:00-04:00,68.402214,68.615775,68.072970,68.313225,7407800,0.0,0.0,0.0
2015-05-21 00:00:00-04:00,68.161943,68.731441,68.161943,68.580170,2930400,0.0,0.0,0.0
2015-05-22 00:00:00-04:00,68.571254,68.722533,68.348794,68.464478,2963800,0.0,0.0,0.0
2015-05-26 00:00:00-04:00,68.411073,68.615739,67.868268,67.975052,5336000,0.0,0.0,0.0


In [ ]:
tradings_bills_df.to_pickle("trading_bills.pkl")

In [162]:
processed_bills_df = pd.read_pickle("trading_bills.pkl")
processed_bills_df

,congress,constitutionalAuthorityStatementText,introducedDate,legislationUrl,number,originChamber,originChamberCode,sponsors,title,type,...,committeeReports,amendments.count,amendments.url,latestAction.actionTime,notes,onBehalfOfSponsor,summary,passed,funds,tradings
0,114,"<pre>[Congressional Record Volume 161, Number ...",2015-06-11,https://www.congress.gov/bill/114th-congress/h...,2734,House,H,"[{'bioguideId': 'B000213', 'district': 6, 'fir...",Do Not Track Kids Act of 2015,HR,...,NaN,NaN,NaN,NaN,NaN,NaN,,False,XLY,Open Hig...
1,115,NaN,2017-03-01,https://www.congress.gov/bill/115th-congress/s...,479,Senate,S,"[{'bioguideId': 'B000944', 'firstName': 'Sherr...",Removing Barriers to Colorectal Cancer Screeni...,S,...,NaN,NaN,NaN,NaN,NaN,NaN,,False,XLV,Open Hig...
2,118,"<pre>\n[Congressional Record Volume 169, Numbe...",2023-05-23,https://www.congress.gov/bill/118th-congress/h...,3603,House,H,"[{'bioguideId': 'C001121', 'district': 6, 'fir...",Small Business Innovation Voucher Act of 2023,HR,...,NaN,NaN,NaN,NaN,NaN,NaN,,False,XLY,Open H...
3,115,<pre>\nFrom the Congressional Record Online th...,2018-12-12,https://www.congress.gov/bill/115th-congress/h...,7289,House,H,"[{'bioguideId': 'S001195', 'district': 8, 'fir...",Fairness for Every Driver Act,HR,...,NaN,NaN,NaN,NaN,NaN,NaN,,False,XLF,Open Hig...
4,117,NaN,2022-06-13,https://www.congress.gov/bill/117th-congress/s...,4380,Senate,S,"[{'bioguideId': 'W000817', 'firstName': 'Eliza...",Military Vehicle Fleet Electrification Act,S,...,NaN,NaN,NaN,NaN,NaN,NaN,,False,XAR,Open H...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,118,NaN,2023-04-26,https://www.congress.gov/bill/118th-congress/s...,1318,Senate,S,"[{'bioguideId': 'K000367', 'firstName': 'Amy',...",Election Worker Protection Act of 2023,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,SPY,Open H...
2483,114,"<pre>[Congressional Record Volume 162, Number ...",2016-02-10,https://www.congress.gov/bill/114th-congress/h...,4528,House,H,"[{'bioguideId': 'L000582', 'district': 33, 'fi...",ENCRYPT Act of 2016,HR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,XAR,Open Hig...
2484,115,"<pre>[Congressional Record Volume 163, Number ...",2017-03-15,https://www.congress.gov/bill/115th-congress/h...,1534,House,H,"[{'bioguideId': 'D000624', 'district': 12, 'fi...",21st Century Worker Opportunity Act,HR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,XLY,Open Hig...
2485,118,"<pre>\n[Congressional Record Volume 170, Numbe...",2024-03-13,https://www.congress.gov/bill/118th-congress/h...,7659,House,H,"[{'bioguideId': 'G000546', 'district': 6, 'fir...",Coast Guard Authorization Act of 2024,HR,...,"[{'citation': 'H. Rept. 118-495', 'url': 'http...",NaN,NaN,NaN,NaN,NaN,NaN,False,XLI,Open H...


In [163]:
processed_bills_df.columns

Index(['congress', 'constitutionalAuthorityStatementText', 'introducedDate',
       'legislationUrl', 'number', 'originChamber', 'originChamberCode',
       'sponsors', 'title', 'type', 'updateDate', 'updateDateIncludingText',
       'cosponsors_d', 'cosponsors_r', 'actions.count', 'actions.url',
       'committees.count', 'committees.url', 'cosponsors.count',
       'cosponsors.countIncludingWithdrawnCosponsors', 'cosponsors.url',
       'latestAction.actionDate', 'latestAction.text', 'policyArea.name',
       'relatedBills.count', 'relatedBills.url', 'subjects.count',
       'subjects.url', 'summaries.count', 'summaries.url',
       'textVersions.count', 'textVersions.url', 'titles.count', 'titles.url',
       'cboCostEstimates', 'laws', 'committeeReports', 'amendments.count',
       'amendments.url', 'latestAction.actionTime', 'notes',
       'onBehalfOfSponsor', 'summary', 'passed', 'funds', 'tradings'],
      dtype='object')

In [164]:
for idx, row in processed_bills_df.iterrows():
    processed_bills_df.at[idx, "bill_id"] = f"{row['type'].lower()}{row['number']}"
processed_bills_df["bill_id"]

0       hr2734
1         s479
2       hr3603
3       hr7289
4        s4380
         ...  
2482     s1318
2483    hr4528
2484    hr1534
2485    hr7659
2486     s1431
Name: bill_id, Length: 2487, dtype: object

In [165]:
processed_bills_df["congress"] = processed_bills_df["congress"].astype(int)
processed_bills_df["congress"]

0       114
1       115
2       118
3       115
4       117
       ... 
2482    118
2483    114
2484    115
2485    118
2486    116
Name: congress, Length: 2487, dtype: int64

In [166]:
processed_bills_df["outcome"] = processed_bills_df["passed"].astype(bool)
processed_bills_df["outcome"]

0       False
1       False
2       False
3       False
4       False
        ...  
2482    False
2483    False
2484    False
2485    False
2486    False
Name: outcome, Length: 2487, dtype: bool

In [167]:
processed_bills_df["num_sponsors"] = processed_bills_df["sponsors"].apply(lambda x: len(x)).astype(int)
processed_bills_df["num_sponsors"]

0       1
1       1
2       1
3       1
4       1
       ..
2482    1
2483    1
2484    1
2485    1
2486    1
Name: num_sponsors, Length: 2487, dtype: int64

In [168]:
processed_bills_df["sponsor_dem_count"] = processed_bills_df["sponsors"].apply(lambda x: sum(1 for s in x if s["party"] == "D")).astype(int)
processed_bills_df["sponsor_rep_count"] = processed_bills_df["sponsors"].apply(lambda x: sum(1 for s in x if s["party"] == "R")).astype(int)
processed_bills_df["sponsor_other_count"] = processed_bills_df["sponsors"].apply(lambda x: sum(1 for s in x if s["party"] not in ["D", "R"])).astype(int)
processed_bills_df["sponsor_bipartisan"] = (processed_bills_df["sponsor_dem_count"] > 0) & (processed_bills_df["sponsor_rep_count"] > 0)
processed_bills_df["sponsor_majority_party"] = processed_bills_df.apply(lambda row: "D" if row["sponsor_dem_count"] > row["sponsor_rep_count"] else ("R" if row["sponsor_rep_count"] > row["sponsor_dem_count"] else "None"), axis=1).astype("category")
processed_bills_df[["sponsor_dem_count", "sponsor_rep_count", "sponsor_other_count", "sponsor_bipartisan", "sponsor_majority_party"]]

,sponsor_dem_count,sponsor_rep_count,sponsor_other_count,sponsor_bipartisan,sponsor_majority_party
0,0,1,0,False,R
1,1,0,0,False,D
2,1,0,0,False,D
3,0,1,0,False,R
4,1,0,0,False,D
...,...,...,...,...,...
2482,1,0,0,False,D
2483,1,0,0,False,D
2484,1,0,0,False,D
2485,0,1,0,False,R


In [169]:
processed_bills_df["num_cosponsors"] = processed_bills_df["cosponsors_d"] + processed_bills_df["cosponsors_r"]
processed_bills_df["bipartisan_support_ratio"] = 0.0
processed_bills_df[["num_cosponsors", "bipartisan_support_ratio"]]

,num_cosponsors,bipartisan_support_ratio
0,14,0.0
1,59,0.0
2,2,0.0
3,0,0.0
4,5,0.0
...,...,...
2482,27,0.0
2483,7,0.0
2484,0,0.0
2485,4,0.0


In [170]:
processed_bills_df["num_committees"] = processed_bills_df["committees.count"]
processed_bills_df["num_actions"] = processed_bills_df["actions.count"]
processed_bills_df["last_action_date"] = processed_bills_df["latestAction.actionDate"]
processed_bills_df[["num_committees", "num_actions", "last_action_date"]]

,num_committees,num_actions,last_action_date
0,1.0,4,2015-06-12
1,1.0,2,2017-03-01
2,1.0,3,2023-05-23
3,1.0,3,2018-12-12
4,1.0,2,2022-06-13
...,...,...,...
2482,1.0,2,2023-04-26
2483,2.0,6,2016-02-29
2484,2.0,6,2017-03-17
2485,2.0,19,2024-05-15


In [171]:
processed_bills_df["days_since_intro"] = processed_bills_df["introducedDate"].apply(lambda x: (pd.Timestamp.now() - pd.to_datetime(x, format='%Y-%m-%d')).days)
processed_bills_df["intro_year"] = pd.to_datetime(processed_bills_df["introducedDate"], format='%Y-%m-%d').dt.year
processed_bills_df["intro_month"] = pd.to_datetime(processed_bills_df["introducedDate"], format='%Y-%m-%d').dt.month
processed_bills_df[["days_since_intro", "intro_year", "intro_month"]]

,days_since_intro,intro_year,intro_month
0,3797,2015,6
1,3168,2017,3
2,894,2023,5
3,2517,2018,12
4,1238,2022,6
...,...,...,...
2482,921,2023,4
2483,3553,2016,2
2484,3154,2017,3
2485,599,2024,3


In [172]:
processed_bills_df["is_house_bill"] = processed_bills_df["type"].str.lower().apply(lambda x: x in ("hr", "hres", "hconres", "hjres"))
processed_bills_df["is_senate_bill"] = processed_bills_df["type"].str.lower().apply(lambda x: x in ("s", "sres", "sconres", "sjres"))
processed_bills_df[["is_house_bill", "is_senate_bill"]]

,is_house_bill,is_senate_bill
0,True,False
1,False,True
2,True,False
3,True,False
4,False,True
...,...,...
2482,False,True
2483,True,False
2484,True,False
2485,True,False


In [173]:
processed_bills_df["has_cosponsors"] = processed_bills_df["num_cosponsors"] > 0
processed_bills_df["total_support"] = processed_bills_df["sponsor_dem_count"] + processed_bills_df["sponsor_rep_count"] + processed_bills_df["num_cosponsors"]
processed_bills_df["strong_bipartisan"] = False
processed_bills_df[["total_support", "has_cosponsors", "strong_bipartisan"]]

,total_support,has_cosponsors,strong_bipartisan
0,15,True,False
1,60,True,False
2,3,True,False
3,1,False,False
4,6,True,False
...,...,...,...
2482,28,True,False
2483,8,True,False
2484,1,False,False
2485,5,True,False


In [174]:
# Calculate returns

def process_tradings(hist):
    if len(hist) == 0:
        return {
            "stock_avg_return_7d": 0.0,
            "stock_avg_return_30d": 0.0,
            "stock_pct_positive": 0.0,
            "stock_volatility": 0.0,
            "stock_momentum": 0.0,
        }
    initial_price = hist["Close"].iloc[0]
    return_7d = 0.0
    return_30d = 0.0
    if len(hist) > 7:
        return_7d = (hist['Close'].iloc[7] / initial_price - 1) * 100
    return_30d = (hist['Close'].iloc[-1] / initial_price - 1) * 100
    daily_returns = hist['Close'].pct_change().dropna()
    return {
        "stock_avg_return_7d": return_7d,
        "stock_avg_return_30d": return_30d,
        "stock_pct_positive": (daily_returns > 0).mean() * 100,
        "stock_volatility": daily_returns.std() * 100,
        "stock_momentum": return_30d - return_7d,
    }

tradings_metrics = processed_bills_df["tradings"].apply(process_tradings)
tradings_metrics_df = pd.json_normalize(tradings_metrics)
processed_bills_df.drop(["stock_avg_return_7d", "stock_avg_return_30d", "stock_pct_positive", "stock_volatility", "stock_momentum"], axis=1, inplace=True, errors='ignore')
tradings_metrics_df
processed_bills_df = pd.concat([processed_bills_df.reset_index(drop=True), tradings_metrics_df.reset_index(drop=True)], axis=1)
processed_bills_df[["stock_avg_return_7d", "stock_avg_return_30d", "stock_pct_positive", "stock_volatility"]]

,stock_avg_return_7d,stock_avg_return_30d,stock_pct_positive,stock_volatility
0,1.718662,1.718662,50.000000,0.543904
1,2.953453,8.039191,75.000000,0.468304
2,-1.330170,2.531388,50.000000,1.174423
3,-1.392023,-7.750174,47.368421,1.549601
4,-0.086085,0.210445,44.444444,1.938386
...,...,...,...,...
2482,2.802115,2.418726,60.000000,0.719555
2483,-3.677046,2.156945,55.555556,1.586494
2484,0.288147,0.852839,45.000000,0.459129
2485,0.057367,2.540568,52.380952,0.690806


In [175]:
processed_bills_df["stock_sector"] = processed_bills_df["funds"].astype("category")
processed_bills_df.drop(list(processed_bills_df.filter(regex='sector_')), inplace=True, errors='ignore')
pd.get_dummies(processed_bills_df["stock_sector"], prefix="sector")
processed_bills_df = pd.concat([processed_bills_df, pd.get_dummies(processed_bills_df["stock_sector"], prefix="sector")], axis=1)
processed_bills_df[["stock_sector", *list(processed_bills_df.filter(regex='sector_'))]]

,stock_sector,sector_ICLN,sector_MOO,sector_SPY,sector_XAR,sector_XLE,sector_XLF,sector_XLI,sector_XLK,sector_XLP,sector_XLRE,sector_XLU,sector_XLV,sector_XLY
0,XLY,False,False,False,False,False,False,False,False,False,False,False,False,True
1,XLV,False,False,False,False,False,False,False,False,False,False,False,True,False
2,XLY,False,False,False,False,False,False,False,False,False,False,False,False,True
3,XLF,False,False,False,False,False,True,False,False,False,False,False,False,False
4,XAR,False,False,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,SPY,False,False,True,False,False,False,False,False,False,False,False,False,False
2483,XAR,False,False,False,True,False,False,False,False,False,False,False,False,False
2484,XLY,False,False,False,False,False,False,False,False,False,False,False,False,True
2485,XLI,False,False,False,False,False,False,True,False,False,False,False,False,False


In [176]:
for col in columns:
    processed_bills_df[col]

In [177]:
processed_bills_df[columns].to_pickle("processed_trading_bills.pkl")

In [178]:
columns = [
    "bill_id",
    "congress",
    "outcome",
    "num_sponsors",
    "sponsor_dem_count",
    "sponsor_rep_count",
    "sponsor_other_count",
    "sponsor_bipartisan",
    "sponsor_majority_party",
    "num_cosponsors",
    "bipartisan_support_ratio",
    "num_committees",
    "num_actions",
    "last_action_date",
    "days_since_intro",
    "intro_year",
    "intro_month",
    "is_house_bill",
    "is_senate_bill",
    "total_support",
    "has_cosponsors",
    "strong_bipartisan",
    "stock_avg_return_7d",
    "stock_avg_return_30d",
    "stock_pct_positive",
    "stock_volatility",
    "stock_momentum",
    "sector_ICLN",
    "sector_MOO",
    "sector_SPY",
    "sector_XAR",
    "sector_XLE",
    "sector_XLF",
    "sector_XLI",
    "sector_XLK",
    "sector_XLP",
    "sector_XLRE",
    "sector_XLU",
    "sector_XLV",
    "sector_XLY",
    "stock_sector"
  ]

In [179]:
pd.read_pickle("processed_trading_bills.pkl")

,bill_id,congress,outcome,num_sponsors,sponsor_dem_count,sponsor_rep_count,sponsor_other_count,sponsor_bipartisan,sponsor_majority_party,num_cosponsors,...,sector_XLE,sector_XLF,sector_XLI,sector_XLK,sector_XLP,sector_XLRE,sector_XLU,sector_XLV,sector_XLY,stock_sector
0,hr2734,114,False,1,0,1,0,False,R,14,...,False,False,False,False,False,False,False,False,True,XLY
1,s479,115,False,1,1,0,0,False,D,59,...,False,False,False,False,False,False,False,True,False,XLV
2,hr3603,118,False,1,1,0,0,False,D,2,...,False,False,False,False,False,False,False,False,True,XLY
3,hr7289,115,False,1,0,1,0,False,R,0,...,False,True,False,False,False,False,False,False,False,XLF
4,s4380,117,False,1,1,0,0,False,D,5,...,False,False,False,False,False,False,False,False,False,XAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,s1318,118,False,1,1,0,0,False,D,27,...,False,False,False,False,False,False,False,False,False,SPY
2483,hr4528,114,False,1,1,0,0,False,D,7,...,False,False,False,False,False,False,False,False,False,XAR
2484,hr1534,115,False,1,1,0,0,False,D,0,...,False,False,False,False,False,False,False,False,True,XLY
2485,hr7659,118,False,1,0,1,0,False,R,4,...,False,False,True,False,False,False,False,False,False,XLI
